In [40]:
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if (!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, httr,jsonlite)

In [42]:
##Hacer lista con urls de los archivos

presupuestos <- list(
  '2024' = 'https://datos.cdmx.gob.mx/dataset/3a6cba97-adb1-4f35-9df1-984cc3657aca/resource/7fa1da81-531c-477c-93db-7580bc1c7ad0/download/egresos_2024_aprobado.csv',
  '2023' = 'https://datos.cdmx.gob.mx/dataset/ea119a03-4f5a-46fa-afbe-bc32bcf79ec3/resource/5137e3db-227e-462d-8d59-1942afbc0c6a/download/egresos_2023_aprobado.csv',
  '2022' = 'https://datos.cdmx.gob.mx/dataset/755f47f1-b937-4131-8a7b-fbe306e747c8/resource/fc93c940-ad75-4e7f-afe6-c624726623a9/download/egresos_2022_aprobado.csv',
  '2021' = 'https://datos.cdmx.gob.mx/dataset/246be014-62ba-46e0-a7ce-9f37c8f01886/resource/682c37bd-96a2-4f78-94bf-3c77c3870f04/download/egresos_2021_aprobado.csv',
  '2020' = 'https://datos.cdmx.gob.mx/dataset/2bb4de63-8212-4d2a-a377-98b10f8384c6/resource/def0fc56-4e26-4cd2-9ec6-200cb86b418e/download/egresos_2020_aprobado.csv',
  '2019' = 'https://datos.cdmx.gob.mx/dataset/8dc52617-63f6-4bfd-9fae-54239675c6d0/resource/9a6f53d7-1751-4ae4-8bd8-d31807be7789/download/egresos_2019_aprobado.csv',
  '2018' = 'https://datos.cdmx.gob.mx/dataset/7a8820f2-3bea-4c69-8b20-1d01b1a99a6a/resource/5b20655c-072f-4347-baee-ab93d930b27b/download/egresos_2018_aprobado.csv'
)

In [43]:
#Función para consultar la api

consulta_api<-function(url){
  r <- GET(url, timeout(10000))
  df <- fromJSON(rawToChar(r$content))
  return(df)
}

In [44]:
presupuestos <- map_dfr(
  names(presupuestos),
  ~ read_csv(presupuestos[[.x]],show_col_types = FALSE)%>%
    mutate(year = as.numeric(.x))%>%
#Identificar presupuesto de innovación, ciencia y tecnología
mutate(ict=case_when((funcion==38) ~ 1, TRUE ~ 0))%>%
group_by(year)%>%
summarise(preup_ict=sum(monto_aprobado[ict == 1]))
)

presupuestos

year,preup_ict
<dbl>,<dbl>
2024,325495931
2023,263752539
2022,65365439
2021,146634666
2020,406148625
2019,359279543
2018,350824762


### Obtener datos de PIB e INPC de INEGI

In [45]:
url_pib<-"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/746106/es/0700/false/BIE/2.0/fe198fe7-c264-840e-c24e-7b28b9af856c?type=json"

url_ipc<-"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/746105,753366/es/0700/false/BIE/2.0/fe198fe7-c264-840e-c24e-7b28b9af856c?type=json"

In [46]:
dfpib<-consulta_api(url_pib)%>%
pluck("Series")%>%
pluck("OBSERVATIONS")%>%
as.data.frame()%>%
janitor::clean_names()%>%
mutate(obs_value=as.numeric(obs_value))%>%
rename(year=time_period,
pib=obs_value
)%>%
mutate(year=as.numeric(year),
pib=as.numeric(pib))%>%
select(year,pib)%>%
filter(year>=2018)%>%
arrange(year)


ipccdmx<-consulta_api(url_ipc)%>%
pluck("Series")%>%
pluck("OBSERVATIONS")%>%
as.data.frame()%>%
janitor::clean_names()%>%
select(time_period_1, obs_value_1)%>%
rename(year=time_period_1,
ipc=obs_value_1)%>%
mutate(year=as.numeric(year),
ipc=as.numeric(ipc))%>%
filter(year>=2018)%>%
arrange(year)

#Pegar los dataframes
dfpib<-dfpib%>%
left_join(ipccdmx, by="year")
dfpib



year,pib,ipc
<dbl>,<dbl>,<dbl>
2018,3694575,100.0000
2019,3679917,102.9339
2020,3293855,107.9443
2021,3496056,111.1398
2022,3650609,117.6172
2023,3806083,123.9455


In [54]:
#Pegar datos a presupuestos
data_presp_pib<-presupuestos%>%
left_join(dfpib, by="year")%>%
#Presupuesto en términos reales
mutate(preup_real=(preup_ict*ipc)/100,
#Prespuesto real en millones de pesos
preup_real=preup_real/1000000,
#Participación del presupuesto en el PIB
participacion=(preup_real/pib)*100)%>%
arrange(year)%>%
filter(year<2024)
data_presp_pib

year,preup_ict,pib,ipc,preup_real,participacion
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018,350824762,3694575,100.0000,350.82476,0.009495673
2019,359279543,3679917,102.9339,369.82053,0.010049697
2020,406148625,3293855,107.9443,438.41448,0.013310071
2021,146634666,3496056,111.1398,162.96949,0.004661524
2022,65365439,3650609,117.6172,76.88099,0.002105977
2023,263752539,3806083,123.9455,326.90933,0.008589128
